In [1]:
with open('app/api_keys.txt') as f:
    lines = f.readlines()

In [2]:
f.close()

In [3]:
lines

['your_api_key_1,your_api_key_2']

In [84]:
import pandas as pd
from scipy.stats import chi2_contingency, shapiro, mannwhitneyu, ttest_ind

def perform_tests(data):
    result = {
        'mann_whitney': [],
        'ttest': [],
        'chi_square': [],
        'shapiro_wilk': []
    }

    death_variable = 'death'
    categorical_columns = [col for col in data.columns if data[col].dtype == 'int64' and col != 'death']
    numerical_columns = [col for col in data.columns if col != 'death' and col not in categorical_columns]

    # Chi-squared test for categorical variables
    for col in categorical_columns:
        contingency_table = pd.crosstab(data[col], data['death'])
        _, p_value, _, _ = chi2_contingency(contingency_table)
        result['chi_square'].append((col, round(p_value, 4)))

    # Shapiro-Wilk test and subsequent t-test or Mann-Whitney U test for numerical variables
    for col in numerical_columns:
        group_0 = data[data['death'] == 0][col]
        group_1 = data[data['death'] == 1][col]

        _, p_value_0 = shapiro(group_0)
        _, p_value_1 = shapiro(group_1)

        result['shapiro_wilk'].append((col, (round(p_value_0, 4), round(p_value_1, 4))))

        if p_value_0 > 0.05 and p_value_1 > 0.05:
            # Use unpaired t-test
            _, p_value = ttest_ind(group_0, group_1, equal_var=False)
            result['ttest'].append((col, round(p_value, 4)))
        else:
            # Use Mann-Whitney U test
            _, p_value = mannwhitneyu(group_0, group_1)
            result['mann_whitney'].append((col, round(p_value, 4)))

    return result